## Matching Records with AWS Lake Formation FindMatches
https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html

AWS Lake Formation provides machine learning capabilities to create custom transforms to cleanse your data. There is currently one available transform named FindMatches. The FindMatches transform enables you to identify duplicate or matching records in your dataset, even when the records do not have a common unique identifier and no fields match exactly. This will not require writing any code or knowing how machine learning works. FindMatches can be useful in many different problems, such as:

Matching Customers: Linking customer records across different customer databases, even when many customer fields do not match exactly across the databases (e.g. different name spelling, address differences, missing or inaccurate data, etc).

Matching Products: Matching products in your catalog against other product sources, such as product catalog against a competitor's catalog, where entries are structured differently.

Improving Fraud Detection: Identifying duplicate customer accounts, determining when a newly created account is (or might be) a match for a previously known fraudulent user.

Other Matching Problems: Match addresses, movies, parts lists, etc etc. In general, if a human being could look at your database rows and determine that they were a match, there is a really good chance that the FindMatches transform can help you.

You can create these transforms when you create a job. The transform that you create is based on a source data store schema and example data that you label (we call this process “teaching” a transform). In this process we generate a file which you label and then upload back which the transform would in a manner learn from). After you teach your transform, you can call it from your Spark-based AWS Glue job (PySpark or Scala Spark) and use it in other scripts with a compatible source data store.

After the transform is created, it is stored in AWS Glue. On the AWS Glue console, you can manage the transforms that you create. On the AWS Glue ML transforms tab, you can edit and continue to teach your machine learning transform. For more information about managing transforms on the console, see [Working with Machine Learning Transforms on the AWS Glue Console](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html).

Note
AWS Glue version 2.0 jobs do not support machine learning transforms. For more information, see [Running Spark ETL Jobs with Reduced Startup Times](https://docs.aws.amazon.com/glue/latest/dg/reduced-start-times-spark-etl-jobs.html).



## Types of Machine Learning Transforms
You can create machine learning transforms to cleanse your data. You can call these transforms from your ETL script. Your data passes from transform to transform in a data structure called a DynamicFrame, which is an extension to an Apache Spark SQL `DataFrame`. The `DynamicFrame` contains your data, and you reference its schema to process your data.

The following types of machine learning transforms are available:

_Find matches_
Finds duplicate records in the source data. You teach this machine learning transform by labeling example datasets to indicate which rows match. The machine learning transform learns which rows should be matches the more you teach it with example labeled data. Depending on how you configure the transform, the output is one of the following:

- A copy of the input table plus a `match_id` column filled in with values that indicate matching sets of records. The `match_id` column is an arbitrary identifier. Any records which have the same `match_id` have been identified as matching to each other. Records with different `match_id`'s do not match.

- A copy of the input table with duplicate rows removed. If multiple duplicates are found, then the record with the lowest primary key is kept.

### Find Matches Transform
You can use the `FindMatches` transform to find duplicate records in the source data. A labeling file is generated or provided to help teach the transform.

#### Getting Started Using the Find Matches Transform
Follow these steps to get started with the `FindMatches` transform:

1. Create a table in the AWS Glue Data Catalog for the source data that is to be cleaned. For information about how to create a crawler, see Working with Crawlers on the AWS Glue Console.

If your source data is a text-based file such as a comma-separated values (CSV) file, consider the following:

- Keep your input record CSV file and labeling file in separate folders. Otherwise, the AWS Glue crawler might consider them as multiple parts of the same table and create tables in the Data Catalog incorrectly.

- Unless your CSV file includes ASCII characters only, ensure that UTF-8 without BOM (byte order mark) encoding is used for the CSV files. Microsoft Excel often adds a BOM in the beginning of UTF-8 CSV files. To remove it, open the CSV file in a text editor, and resave the file as UTF-8 without BOM.

2. On the AWS Glue console, create a job, and choose the Find matches transform type.

Important
The data source table that you choose for the job can't have more than 100 columns.

3. Tell AWS Glue to generate a labeling file by choosing Generate labeling file. AWS Glue takes the first pass at grouping similar records for each labeling_set_id so that you can review those groupings. You label matches in the label column.

- If you already have a labeling file, that is, an example of records that indicate matching rows, upload the file to Amazon Simple Storage Service (Amazon S3). For information about the format of the labeling file, see Labeling File Format. Proceed to step 4.

4. Download the labeling file and label the file as described in the Labeling section.

5. Upload the corrected labelled file. AWS Glue runs tasks to teach the transform how to find matches.

On the __Machine learning transforms__ list page, choose the __History__ tab. This page indicates when AWS Glue performs the following tasks:

- __Import labels__
- __Export labels__
- __Generate labels__
- __Estimate quality__

To create a better transform, you can iteratively download, label, and upload the labelled file. In the initial runs, a lot more records might be mismatched. But AWS Glue learns as you continue to teach it by verifying the labeling file.

Evaluate and tune your transform by evaluating performance and results of finding matches. For more information, see [Tuning Machine Learning Transforms in AWS Glue](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html).

### Labeling

When `FindMatches` generates a labeling file, records are selected from your source table. Based on previous training, `FindMatches` identifies the most valuable records to learn from.

The act of labeling is editing a labeling file (we suggest using a spreadsheet such as Microsoft Excel) and adding identifiers, or labels, into the label column that identifies matching and nonmatching records. It is important to have a clear and consistent definition of a match in your source data. FindMatches learns from which records you designate as matches (or not) and uses your decisions to learn how to find duplicate records.

When a labeling file is generated by `FindMatches`, approximately 100 records are generated. These 100 records are typically divided into 10 labeling sets, where each labeling set is identified by a unique labeling_set_id generated by `FindMatches`. Each labeling set should be viewed as a separate labeling task independent of the other labeling sets. Your task is to identify matching and non-matching records within each labeling set.

#### Tips for Editing Labeling Files in a Spreadsheet
When editing the labeling file in a spreadsheet application, consider the following:

The file might not open with column fields fully expanded. You might need to expand the `labeling_set_id` and `label` columns to see content in those cells.

If the primary key column is a number, such as a long data type, the spreadsheet might interpret it as a number and change the value. This key value must be treated as text. To correct this problem, format all the cells in the primary key column as Text data.

### Labeling File Format
The labeling file that is generated by AWS Glue to teach your `FindMatches` transform uses the following format. If you generate your own file for AWS Glue, it must follow this format as well:

- It is a comma-separated values (CSV) file.

- It must be encoded in `UTF-8`. If you edit the file using Microsoft Windows, it might be encoded with `cp1252`.

- It must be in an Amazon S3 location to pass it to AWS Glue.

- Use a moderate number of rows for each labeling task. 10–20 rows per task are recommended, although 2–30 rows per task are acceptable. Tasks larger than 50 rows are not recommended and may cause poor results or system failure.

- If you have already-labeled data consisting of pairs of records labeled as a "match" or a "no-match", this is fine. These labeled pairs can be represented as labeling sets of size 2. In this case label both records with, for instance, a letter "A" if they match, but label one as "A" and one as "B" if they do not match.

Note
Because it has additional columns, the labeling file has a different schema from a file that contains your source data. Place the labeling file in a different folder from any transform input CSV file so that the AWS Glue crawler does not consider it when it creates tables in the Data Catalog. Otherwise, the tables created by the AWS Glue crawler might not correctly represent your data.

- The first two columns (`labeling_set_id`, `label`) are required by AWS Glue. The remaining columns must match the schema of the data that is to be processed.

- For each `labeling_set_id`, you identify all matching records by using the same label. A label is a unique string placed in the `label` column. We recommend using labels that contain simple characters, such as A, B, C, and so on. Labels are case sensitive and are entered in the label column.

- Rows that contain the same `labeling_set_id` and the same label are understood to be labeled as a match.

- Rows that contain the same `labeling_set_id` and a different label are understood to be labeled as not a match

- Rows that contain a different `labeling_set_id` are understood to be conveying no information for or against matching.

The following is an example of labeling the data:

|labeling_set_id|label|first_name|last_name|Birthday|
|:---|:---:|:---:|:---:|---:|
|ABC123|A|John|Doe|04/01/1980|
|ABC123|B|Jane|Smith|04/03/1980|
|ABC123|A|Johnny|Doe|04/01/1980|
|ABC123|A|Jon|Doe|04/01/1980|
|DEF345|A|Richard|Jones|12/11/1992|
|DEF345|A|Rich|Jones|11/12/1992|
|DEF345|B|Sarah|Jones|12/11/1992|
|DEF345|C|Richie|Jones Jr.|05/06/2017|
|DEF345|B|Sarah|Jones-Walker|12/11/1992|
|GHI678|A|Robert|Miller|1/3/1999|
|GHI678|A|Bob|Miller|1/3/1999|
|XYZABC|A|William|Robinson|2/5/2001|
|XYZABC|B|Andrew|Robinson|2/5/1971|

- In the above example we identify John/Johnny/Jon Doe as being a match and we teach the system that these records do not match Jane Smith. Separately, we teach the system that Richard and Rich Jones are the same person, but that these records are not a match to Sarah Jones/Jones-Walker and Richie Jones Jr.

- As you can see, the scope of the labels is limited to the labeling_set_id. So labels do not cross labeling_set_id boundaries. For example, a label "A" in labeling_set_id 1 does not have any relation to label "A" in labeling_set_id 2.

- If a record does not have any matches within a labeling set, then assign it a unique label. For instance, Jane Smith does not match any record in labeling set ABC123, so it is the only record in that labeling set with the label of B.

- The labeling set "GHI678" shows that a labeling set can consist of just two records which are given the same label to show that they match. Similarly, "XYZABC" shows two records given different labels to show that they do not match.

- Note that sometimes a labeling set may contain no matches (that is, you give every record in the labeling set a different label) or a labeling set might all be "the same" (you gave them all the same label). This is okay as long as your labeling sets collectively contain examples of records that are and are not "the same" by your criteria.

Important
Confirm that the IAM role that you pass to AWS Glue has access to the Amazon S3 bucket that contains the labeling file. By convention, AWS Glue policies grant permission to Amazon S3 buckets or folders whose names are prefixed with __aws-glue-__. If your labeling files are in a different location, add permission to that location in the IAM role.

## Tuning Machine Learning Transforms in AWS Glue

You can tune your machine learning transforms in AWS Glue to improve the results of your data-cleansing jobs to meet your objectives. To improve your transform, you can teach it by generating a labeling set, adding labels, and then repeating these steps several times until you get your desired results. You can also tune by changing some machine learning parameters.

For more information about machine learning transforms, see [Matching Records with AWS Lake Formation FindMatches](https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html).

Topics

- [Machine Learning Measurements](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html#machine-learning-terminology)
- [Deciding Between Precision and Recall](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html#machine-learning-precision-recall-tradeoff)
- [Deciding Between Accuracy and Cost](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html#machine-learning-accuracy-cost-tradeoff)
- [Teaching the Find Matches Transform](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html#machine-learning-teaching)


### Machine Learning Measurements
To understand the measurements that are used to tune your machine learning transform, you should be familiar with the following terminology:

__True positive (TP)__
    A match in the data that the transform correctly found, sometimes called a hit.

__True negative (TN)__
    A nonmatch in the data that the transform correctly rejected.

__False positive (FP)__
    A nonmatch in the data that the transform incorrectly classified as a match, sometimes called a false alarm.

__False negative (FN)__
    A match in the data that the transform didn't find, sometimes called a miss.

For more information about the terminology that is used in machine learning, see [Confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) in Wikipedia.

To tune your machine learning transforms, you can change the value of the following measurements in the __Advanced properties__ of the transform.

__Precision__ measures how well the transform finds true positives from the total true positives possible. For more information, see [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) in Wikipedia.

__Recall__ measures how well the transform finds true positives from the total records in the source data. For more information, see [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) in Wikipedia.

__Accuracy__ measures how well the transform finds true positives and true negatives. Increasing accuracy requires more machine resources and cost. But it also results in increased recall. For more information, see [Accuracy and precision](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_information_systems) in Wikipedia.

__Cost__ measures how many compute resources (and thus money) are consumed to run the transform.

### Deciding Between Precision and Recall
Each `FindMatches transform` contains a `precision-recall` parameter. You use this parameter to specify one of the following:

If you are more concerned about the transform falsely reporting that two records match when they actually don't match, then you should emphasize precision.

If you are more concerned about the transform failing to detect records that really do match, then you should emphasize recall.

You can make this trade-off on the AWS Glue console or by using the AWS Glue machine learning API operations.

#### When to Favor Precision

Favor precision if you are more concerned about the risk that `FindMatches` results in a pair of records matching when they don't actually match. To favor precision, choose a higher `precision-recall` trade-off value. With a higher value, the `FindMatches` transform requires more evidence to decide that a pair of records should be matched. The transform is tuned to bias toward saying that records do not match.

For example, suppose that you're using `FindMatches` to detect duplicate items in a video catalog, and you provide a higher `precision-recall` value to the transform. If your transform incorrectly detects that Star Wars: A New Hope is the same as Star Wars: The Empire Strikes Back, a customer who wants A New Hope might be shown The Empire Strikes Back. This would be a poor customer experience.

However, if the transform fails to detect that Star Wars: A New Hope and Star Wars: Episode IV—A New Hope are the same item, the customer might be confused at first but might eventually recognize them as the same. It would be a mistake, but not as bad as the previous scenario.

#### When to Favor Recall

Favor recall if you are more concerned about the risk that the `FindMatches` transform results might fail to detect a pair of records that actually do match. To favor recall, choose a lower `precision-recall` trade-off value. With a lower value, the FindMatches transform requires less evidence to decide that a pair of records should be matched. The transform is tuned to bias toward saying that records do match.

For example, this might be a priority for a security organization. Suppose that you are matching customers against a list of known defrauders, and it is important to determine whether a customer is a defrauder. You are using `FindMatches` to match the defrauder list against the customer list. Every time `FindMatches` detects a match between the two lists, a human auditor is assigned to verify that the person is, in fact, a defrauder. Your organization might prefer to choose recall over precision. In other words, you would rather have the auditors manually review and reject some cases when the customer is not a defrauder than fail to identify that a customer is, in fact, on the defrauder list.

#### How to Favor Both Precision and Recall

The best way to improve both precision and recall is to label more data. As you label more data, the overall accuracy of the FindMatches transform improves, thus improving both precision and recall. Nevertheless, even with the most accurate transform, there is always a gray area where you need to experiment with favoring precision or recall, or choose a value in the middle.

### Deciding Between Accuracy and Cost

Each `FindMatches` transform contains an `accuracy-cost` parameter. You can use this parameter to specify one of the following:

If you are more concerned with the transform accurately reporting that two records match, then you should emphasize accuracy.

If you are more concerned about the cost or speed of running the transform, then you should emphasize lower cost.

You can make this trade-off on the AWS Glue console or by using the AWS Glue machine learning API operations.

#### When to Favor Accuracy

Favor accuracy if you are more concerned about the risk that the `find matches` results won't contain matches. To favor accuracy, choose a higher `accuracy-cost` trade-off value. With a higher value, the `FindMatches` transform requires more time to do a more thorough search for correctly matching records. Note that this parameter doesn't make it less likely to falsely call a nonmatching record pair a match. The transform is tuned to bias towards spending more time finding matches.

#### When to Favor Cost

Favor cost if you are more concerned about the cost of running the find matches transform and less about how many matches are found. To favor cost, choose a lower `accuracy-cost` trade-off value. With a lower value, the `FindMatches` transform requires fewer resources to run. The transform is tuned to bias towards finding fewer matches. If the results are acceptable when favoring lower cost, use this setting.

#### How to Favor Both Accuracy and Lower Cost

It takes more machine time to examine more pairs of records to determine whether they might be matches. If you want to reduce cost without reducing quality, here are some steps you can take:

- Eliminate records in your data source that you aren't concerned about matching.

- Eliminate columns from your data source that you are sure aren't useful for making a match/no-match decision. A good way of deciding this is to eliminate columns that you don't think affect your own decision about whether a set of records is “the same.”

### Teaching the Find Matches Transform
Each `FindMatches` transform must be taught what should be considered a match and what should not be considered a match. You teach your transform by adding labels to a file and uploading your choices to AWS Glue.

You can orchestrate this labeling on the AWS Glue console or by using the AWS Glue machine learning API operations.

#### How Many Times Should I Add Labels? How Many Labels Do I Need?

The answers to these questions are mostly up to you. You must evaluate whether `FindMatches` is delivering the level of accuracy that you need and whether you think the extra labeling effort is worth it for you. The best way to decide this is to look at the “Precision,” “Recall,” and “Area under the precision recall curve” metrics that you can generate when you choose Estimate quality on the AWS Glue console. After you label more sets of tasks, rerun these metrics and verify whether they have improved. If, after labeling a few sets of tasks, you don't see improvement on the metric that you are focusing on, the transform quality might have reached a plateau.

#### Why Are Both True Positive and True Negative Labels Needed?

The `FindMatches` transform needs both positive and negative examples to learn what you think is a match. If you are labeling FindMatches-generated training data (for example, using the I do not have labels option), `FindMatches` tries to generate a set of “label set ids” for you. Within each task, you give the same “label” to some records and different “labels” to other records. In other words, the tasks generally are not either all the same or all different (but it's okay if a particular task is all “the same” or all “not the same”).

If you are teaching your `FindMatches` transform using the Upload labels from S3 option, try to include both examples of matching and nonmatching records. It's acceptable to have only one type. These labels help you build a more accurate FindMatches transform, but you still need to label some records that you generate using the Generate labeling file option.

#### How Can I Enforce That the Transform Matches Exactly as I Taught It?

The `FindMatches` transform learns from the labels that you provide, so it might generate records pairs that don't respect the provided labels. To enforce that the `FindMatches` transform respects your labels, select __EnforceProvidedLabels__ in __FindMatchesParameter__.

#### What Techniques Can You Use When an ML Transform Identifies Items as Matches That Are Not True Matches?

You can use the following techniques:

Increase the `precisionRecallTradeoff` to a higher value. This eventually results in finding fewer matches, but it should also break up your big cluster when it reaches a high enough value.

- Take the output rows corresponding to the incorrect results and reformat them as a labeling set (removing the match_id column and adding a labeling_set_id and label column). If necessary, break up (subdivide) into multiple labeling sets to ensure that the labeler can keep each labeling set in mind while assigning labels. Then, correctly label the matching sets and upload the label file and append it to your existing labels. This might teach your transformer enough about what it is looking for to understand the pattern.

- (Advanced) Finally, look at that data to see if there is a pattern that you can detect that the system is not noticing. Preprocess that data using standard AWS Glue functions to normalize the data. Highlight what you want the algorithm to learn from by separating data that you know to be differently important into their own columns. Or construct combined columns from columns whose data you know to be related.

## Working with Machine Learning Transforms on the AWS Glue Console
https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html

You can use AWS Glue to create custom machine learning transforms that can be used to cleanse your data. You can use these transforms when you create a job on the AWS Glue console.

For information about how to create a machine learning transform, see [Matching Records with AWS Lake Formation FindMatches](https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html).

Topics

- [Transform Properties](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html#console-machine-learning-properties)
- [Adding and Editing Machine Learning Transforms](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html#console-machine-learning-transforms-actions)
- [Viewing Transform Details](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html#console-machine-learning-transforms-details)


### Transform Properties
To view an existing machine learning transform, sign in to the AWS Management Console, and open the AWS Glue console at https://console.aws.amazon.com/glue/. Then choose the __ML transforms tab__.

The __Machine Learning Transforms__ list displays the following properties for each transform:

__Transform name__
The unique name you gave the transform when you created it.

__Transform ID__
A unique identifier of the transform.

__Type__
The type of machine learning transform; for example, __Find matching records__.

__AWS Glue version__
This value determines which version of AWS Glue this machine learning transform is compatible with. AWS Glue 1.0 is recommended for most customers. If the value is not set, the AWS Glue compatibility defaults to AWS Glue 0.9. For more information, see AWS Glue Versions.

__Status__
Indicates whether the transform is Ready or Needs teaching. To run a machine learning transform successfully in a job, it must be Ready.

When you create a FindMatches transform, you specify the following configuration information:

__Primary key__
The name of a column that uniquely identifies rows in the source table.

__Type__
The type of machine learning transform; for example, __Find matches__.

__Merge matching records__
Indicates whether the transform is to remove duplicates in the target. The record with the lowest primary key value is written to the output of the transform.

### Adding and Editing Machine Learning Transforms
You can view, delete, set up and teach, or tune a transform on the AWS Glue console. Select the check box next to the transform in the list, choose __Action__, and then choose the action that you want to take.

To add a new machine learning transform, choose the __Jobs__ tab, and then choose __Add job__. Follow the instructions in the __Add job__ wizard to add a job with a machine learning transform such as `FindMatches`. For more information, see [Matching Records with AWS Lake Formation `FindMatches`](https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html).

### Viewing Transform Details
Transform details include the information that you defined when you created the transform. To view the details of a transform, select the __transform in the Machine learning transforms list__, and review the information on the following tabs:

- History
- Details
- Estimate quality

### History
The History tab shows your transform task run __history__. Several types of tasks are run to teach a transform. For each task, the run metrics include the following:

- __Run ID__ is an identifier created by AWS Glue for each run of this task.
- __Task type__ shows the type of task run.
- __Status__ shows the success of each task listed with the most recent run at the top.
- __Error__ shows the details of an error message if the run was not successful.
- __Start time__ shows the date and time (local time) that the task started.
- __Execution time__ shows the length of time during which the job run consumed resources. The amount is calculated from when the job run starts consuming resources until it finishes.
- __Last modified__ shows the date and time (local time) that the task was last modified.
- __Logs__ links to the logs written to stdout for this job run.

    The __Logs__ link takes you to Amazon CloudWatch Logs. There you can view the details about the tables that were created in the AWS Glue Data Catalog and any errors that were encountered. You can manage your log retention period on the CloudWatch console. The default log retention is `Never Expire`. For more information about how to change the retention period, see [Change Log Data Retention in CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/Working-with-log-groups-and-streams.html#SettingLogRetention) in the *Amazon CloudWatch* Logs User Guide.

- __Error__ logs links to the logs written to `stderr` for this task run.

This link takes you to CloudWatch Logs, where you can see details about any errors that were encountered. You can manage your log retention period on the CloudWatch console. The default log retention is `Never Expire`. For more information about how to change the retention period, see [Change Log Data Retention in CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/Working-with-log-groups-and-streams.html#SettingLogRetention) in the Amazon CloudWatch Logs User Guide.

- __Download label file__ shows a link to Amazon S3 for a generated labeling file.

### Details
The __Details__ tab includes attributes of your transform. It shows you the details about the transform definition, including the following:

- __Transform name__ shows the name of the transform.
- __Type__ lists the type of the transform.
- __Status__ displays whether the transform is ready to be used in a script or job.
- __Force output to match labels__ displays whether the transform forces the output to match the labels provided by the user.

### Estimate quality
The __Estimate quality__ tab shows the metrics that you use to measure the quality of the transform. Estimates are calculated by comparing the transform match predictions using a subset of your labeled data against the labels you have provided. These estimates are approximate.You can invoke an __Estimate quality__ task run from this tab.

The __Estimate quality__ tab shows the metrics from the last __Estimate quality__ run including the following properties:

- __Area under the Precision-Recall curve__ is a single number estimating the upper bound of the overall quality of the transform. It is independent of the choice made for the precision-recall parameter. Higher values indicate that you have a more attractive precision-recall tradeoff.

- __Precision__ estimates how often the transform is correct when it predicts a match.

- __Recall upper limit__ estimates that for an actual match, how often the transform predicts the match.

- __Max F1__ estimates the transform's accuracy between 0 and 1, where 1 is the best accuracy. For more information, see [F1 score](https://en.wikipedia.org/wiki/F1_score) in Wikipedia.

For information about understanding quality estimates versus true quality, see [Quality Estimates Versus End-to-End (True) Quality](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html#console-machine-learning-quality-estimates-true-quality).

For more information about tuning your transform, see [Tuning Machine Learning Transforms in AWS Glue](https://docs.aws.amazon.com/glue/latest/dg/add-job-machine-learning-transform-tuning.html).

### Quality Estimates Versus End-to-End (True) Quality
In the `FindMatches` machine learning transform, AWS Glue estimates the quality of your transform by presenting the internal machine-learned model with a number of pairs of records that you provided matching labels for but that the model has not seen before. These quality estimates are a function of the quality of the machine-learned model (which is influenced by the number of records that you label to “teach” the transform). The end-to-end, or true recall (which is not automatically calculated by the `FindMatches` transform) is also influenced by the `FindMatches` filtering mechanism that proposes a wide variety of possible matches to the machine-learned model.

You can tune this filtering method primarily by using the __Lower Cost-Accuracy__ slider. As you move this slider closer to the __Accuracy__ end, the system does a more thorough and expensive search for pairs of records that might be matches. More pairs of records are fed to your machine-learned model, and your `FindMatches` transform's end-to-end or true recall gets closer to the estimated recall metric. As a result, changes in the end-to-end quality of your matches as a result of changes in your matches's cost/accuracy tradeoff will typically not be reflected in the quality estimate.
